Sure, let's create a simplified version of this application that calculates and displays the max profit and max loss for an option. We'll need the option type (call or put), strike price, and current stock price as inputs.

Here's a step-by-step plan:

Create a new PyQt5 application with a simpler UI.
The UI will have input fields for the option type, strike price, and current stock price.
It will also have labels to display the max profit and max loss.
When the input fields are updated, the application will calculate the max profit and max loss based on the input values.
The max profit for a call option is theoretically unlimited, while for a put option it's the strike price minus the option premium (assuming the option premium is zero for simplicity). The max loss for both types is the option premium (again, assuming it's zero for simplicity).
The application will update the labels with the calculated max profit and max loss.
Here's the Python code for this application

In [2]:
from PyQt5 import QtWidgets, QtCore
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import numpy as np
from matplotlib.ticker import FuncFormatter

class MyApp(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        self.setWindowTitle("Profit/Loss Chart")

        # Create the form layout
        form_layout = QtWidgets.QVBoxLayout()
        
        # Create the slider
        self.slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(100)
        self.slider.setValue(50)

        # Create the form
        self.current_stock_price_input = QtWidgets.QLineEdit()
        self.strike_price_input1 = QtWidgets.QLineEdit()
        self.strike_price_input2 = QtWidgets.QLineEdit()
        self.credit_debit_input = QtWidgets.QLineEdit()

        # Add form fields to the layout
        form_layout.addWidget(QtWidgets.QLabel("Current Stock Price:"))
        form_layout.addWidget(self.current_stock_price_input)
        form_layout.addWidget(QtWidgets.QLabel("Strike Price 1:"))
        form_layout.addWidget(self.strike_price_input1)
        form_layout.addWidget(QtWidgets.QLabel("Strike Price 2:"))
        form_layout.addWidget(self.strike_price_input2)
        form_layout.addWidget(QtWidgets.QLabel("Credit/Debit:"))
        form_layout.addWidget(self.credit_debit_input)

        # Create the plot
        self.figure = Figure()
        self.canvas = FigureCanvas(self.figure)
        self.ax = self.figure.add_subplot(111)
        self.ax.set_xlabel('Stock Price')
        self.ax.set_ylabel('Profit/Loss')
        self.ax.set_ylim([-1, 1])
        
        # Add padding to the bottom of the plot
        self.figure.subplots_adjust(bottom=0.2)

        # Add the plot to the layout
        form_layout.addWidget(self.canvas)
        
        # Add the slider to the layout
        form_layout.addWidget(self.slider)
        
        # Connect the slider to the update_line method
        self.slider.valueChanged.connect(self.update_line)

        # Set the layout
        self.widget = QtWidgets.QWidget()
        self.widget.setLayout(form_layout)
        self.setCentralWidget(self.widget)

        # Connect the input fields to the update_chart method
        self.current_stock_price_input.textChanged.connect(self.update_chart)
        self.strike_price_input1.textChanged.connect(self.update_chart)
        self.strike_price_input2.textChanged.connect(self.update_chart)
        self.credit_debit_input.textChanged.connect(self.update_chart)

        self.show()

    def update_chart(self):
        current_stock_price = float(self.current_stock_price_input.text()) if self.current_stock_price_input.text() else 0.0
        strike_price1 = float(self.strike_price_input1.text()) if self.strike_price_input1.text() else 0.0
        strike_price2 = float(self.strike_price_input2.text()) if self.strike_price_input2.text() else 0.0
        credit_debit = float(self.credit_debit_input.text()) if self.credit_debit_input.text() else 0.0


        # Calculate the range of stock prices
        prices = np.linspace(0.95 * strike_price1, 1.05 * strike_price2, 100)
        
        # Calculate the midpoint between the two strike prices
        break_even_price = (strike_price1 + strike_price2) / 2

        # Set the initial slider value and price vertical line to the midpoint
        denominator = (1.05 * strike_price2 - 0.95 * strike_price1)
        if denominator != 0:
            self.slider.setValue((break_even_price - 0.95 * strike_price1) / denominator * 100)
        else:
            self.slider.setValue(50)
        
        # Update the plot
        self.ax.clear()
        self.ax.plot(prices, np.zeros_like(prices))  # profit/loss is zero for all prices
        self.line = self.ax.axvline(x=break_even_price, color='b')  # add vertical line for break_even_price
        self.ax.axvline(x=strike_price1, color='r')  # add vertical line for strike price 1
        self.ax.axvline(x=strike_price2, color='g')  # add vertical line for strike price 2
        self.ax.set_xlabel('Stock Price')
        self.ax.set_ylabel('Profit/Loss')
        self.ax.set_ylim([-1, 1])
        
        # Set x-axis limits to zoom in
        self.ax.set_xlim([0.98 * strike_price1, 1.02 * strike_price2])
        
        # Format x-axis labels to 2 decimal places
        self.ax.xaxis.set_major_formatter(FuncFormatter('{:.2f}'.format))
        
        self.canvas.draw()
        
    def update_line(self, value):
        # Convert the slider value back to a stock price
        strike_price1 = float(self.strike_price_input1.text()) if self.strike_price_input1.text() else 0.0
        strike_price2 = float(self.strike_price_input2.text()) if self.strike_price_input2.text() else 0.0
        current_stock_price = 0.95 * strike_price1 + value / 100 * (1.05 * strike_price2 - 0.95 * strike_price1)

        # Update the position of the line
        self.line.set_xdata(current_stock_price)
        
        
        # Remove the old label if it exists
        if hasattr(self, 'label'):
            self.label.remove()

        # Add a label to the top of the line
        self.label = self.ax.annotate(f'{current_stock_price:.2f}', 
                                    xy=(current_stock_price, 1), 
                                    xytext=(0, 5), 
                                    textcoords='offset points', 
                                    ha='right', 
                                    va='bottom')

        # Redraw the canvas
        self.canvas.draw()

if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    myapp = MyApp()
    app.exec_()

/var/folders/b6/jx3jtxg13qd2m5wc8jr_4qsc0000gn/T/ipykernel_31020/3133945053.py:105: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  self.ax.set_xlim([0.98 * strike_price1, 1.02 * strike_price2])
/var/folders/b6/jx3jtxg13qd2m5wc8jr_4qsc0000gn/T/ipykernel_31020/3133945053.py:90: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.slider.setValue((break_even_price - 0.95 * strike_price1) / denominator * 100)
/var/folders/b6/jx3jtxg13qd2m5wc8jr_4qsc0000gn/T/ipykernel_31020/3133945053.py:119: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  self.line.set_xdata(current_stock_price)


ValueError: could not convert string to float: '.'

: 